In [1]:
import os
from influence_moo.config import load_config

In [2]:
top_dir = "~/influence-multi-objective/results/captain_d/redo2"
batch_dir_root = '~/influence-multi-objective/sbatch/sep_30_2024/'

In [3]:
def contractuser(path, user):
    return str.replace(path, user, "~")

config_dirs = []
for dirpath, _, filenames in os.walk(os.path.expanduser(top_dir)):
    for filename in filenames:
        if filename == 'config.yaml':
            config_dir = contractuser(dirpath, '/home/gonzaeve')+'/config.yaml'
            config_dirs.append(config_dir)


In [4]:
for c in config_dirs:
    print(c)

~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Traj/config.yaml
~/influence-multi-objective/results/captain_d/redo2/fitness_critic/config.yaml
~/influence-multi-objective/results/captain_d/redo2/D/config.yaml
~/influence-multi-objective/results/captain_d/redo2/G/config.yaml
~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Step/config.yaml
~/influence-multi-objective/results/captain_d/redo2/alignment/config.yaml


In [5]:
commands = []
for config_dir in config_dirs:
    command_start = 'python ~/influence-multi-objective/influence_moo/influence_moo/run_cli.py '
    config = load_config(config_dir)
    num_trials = config['experiment']['num_trials']
    for t in range(num_trials):
        command = command_start + config_dir + ' -t ' + str(t)
        commands.append(command)

In [6]:
for c in commands:
    print(c)

python ~/influence-multi-objective/influence_moo/influence_moo/run_cli.py ~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Traj/config.yaml -t 0
python ~/influence-multi-objective/influence_moo/influence_moo/run_cli.py ~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Traj/config.yaml -t 1
python ~/influence-multi-objective/influence_moo/influence_moo/run_cli.py ~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Traj/config.yaml -t 2
python ~/influence-multi-objective/influence_moo/influence_moo/run_cli.py ~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Traj/config.yaml -t 3
python ~/influence-multi-objective/influence_moo/influence_moo/run_cli.py ~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Traj/config.yaml -t 4
python ~/influence-multi-objective/influence_moo/influence_moo/run_cli.py ~/influence-multi-objective/results/captain_d/redo2/D-Indirect-Traj/config.yaml -t 5
python ~/influence-multi-objective/influence_m

In [7]:
file_dirs = []

spacer = 'trials'
batch_dir = batch_dir_root+spacer

for c in commands:
    experiment_name = c.split(' ')[2].replace('/', '.').replace('~.influence-multi-objective.results.', '').replace('.config.yaml','')
    trial_num = c.split(' ')[-1]
    file_name = experiment_name +'.'+trial_num+'.sh'
    file_dir = batch_dir + '/' + file_name
    file_dirs.append(file_dir)

In [8]:
for f in file_dirs:
    print(f)

~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.0.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.1.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.2.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.3.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.4.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.5.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.6.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.7.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.8.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.D-Indirect-Traj.9.sh
~/influence-multi-objective/sbatch/sep_30_2024/trials/captain_d.redo2.fitness_critic.0.sh


In [9]:
# Define the string you want to write
file_str_start = \
"""#!/bin/bash
#SBATCH --time=1-00:00:00
#SBATCH --constraint=skylake
#SBATCH --mem=8G
#SBATCH -c 4

module load python/3.10
source ~/venv/influence/bin/activate

"""
if not os.path.exists(os.path.expanduser(batch_dir)):
    os.makedirs(os.path.expanduser(batch_dir))
for c, f in zip(commands, file_dirs):
    file_str = file_str_start + '\n' + c + '\n'
    with open(os.path.expanduser(f), 'w') as file:
        file.write(file_str)

In [10]:
batch_commands = []

for f in file_dirs:
    batch_commands.append('sbatch '+f)

batch_file_str = '\n'.join(batch_commands)+'\n'
with open(os.path.expanduser(batch_dir_root+'/sbatch.sh'), 'w') as file:
    file.write(batch_file_str)